# DTOcean Moorings and Foundation with Reliability Test

In [28]:
%matplotlib inline

In [29]:
from IPython.display import display, HTML

In [30]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14.0, 8.0)

In [31]:
import numpy as np

In [32]:
from dtocean_core import start_logging
from dtocean_core.core import Core
from dtocean_core.menu import ModuleMenu, ProjectMenu, ThemeMenu
from dtocean_core.pipeline import Tree

In [33]:
def html_list(x):
    message = "<ul>"
    for name in x:
        message += "<li>{}</li>".format(name)
    message += "</ul>"
    return message
def html_dict(x):
    message = "<ul>"
    for name, status in x.iteritems():
        message += "<li>{}: <b>{}</b></li>".format(name, status)
    message += "</ul>"
    return message

In [34]:
# Bring up the logger
start_logging()

2016-12-06 20:01:43,546 - INFO - dtocean_core - Begin logging for dtocean_core.


## Create the core, menus and pipeline tree

The core object carrys all the system information and is operated on by the other classes

In [35]:
new_core = Core()
project_menu = ProjectMenu()
module_menu = ModuleMenu()
theme_menu = ThemeMenu()
pipe_tree = Tree()

## Create a new project

In [36]:
project_title = "DTOcean"  
new_project = project_menu.new_project(new_core, project_title)

2016-12-06 20:01:44,907 - INFO - aneris.entity.simulation - Created new Simulation with title "Default"
2016-12-06 20:01:44,907 - INFO - aneris.control.simulation - Datastate with level "initial" stored
2016-12-06 20:01:44,907 - INFO - aneris.control.pipeline - New Hub created for interface ProjectInterface.


## Set the device type

In [37]:
options_branch = pipe_tree.get_branch(new_core, new_project, "System Type Selection")
variable_id = "device.system_type" 
my_var = options_branch.get_input_variable(new_core, new_project, variable_id)
my_var.set_raw_interface(new_core, "Tidal Floating")
my_var.read(new_core, new_project)

2016-12-06 20:01:44,937 - INFO - aneris.control.data - New "device.system_type" data stored with index D7Y3OC
2016-12-06 20:01:44,937 - INFO - aneris.control.simulation - Datastate stored


## Initiate the pipeline
This step will be important when the database is incorporated into the system as it will effect the operation of the pipeline.

In [38]:
project_menu.initiate_pipeline(new_core, new_project)

2016-12-06 20:01:44,947 - INFO - aneris.control.simulation - Datastate with level "system type selection start" stored
2016-12-06 20:01:44,957 - INFO - aneris.control.data - New "hidden.pipeline_active" data stored with index D0W7VC
2016-12-06 20:01:44,957 - INFO - aneris.control.simulation - Datastate with level "system type selection output" stored
2016-12-06 20:01:44,957 - INFO - aneris.control.pipeline - New Pipeline created for interface ModuleInterface.
2016-12-06 20:01:44,957 - INFO - aneris.control.pipeline - New Hub created for interface ThemeInterface.


## Discover available modules

In [39]:
names = module_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

## Activate a module

Note that the order of activation is important and that we can't deactivate yet!

In [40]:
module_name = 'Mooring and Foundations'
module_menu.activate(new_core, new_project, module_name)

## Check the status of the module inputs

In [41]:
mooring_branch = pipe_tree.get_branch(new_core, new_project, 'Mooring and Foundations')
input_status = mooring_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Activate Reliability Theme

In [42]:
names = theme_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

In [43]:
theme_menu.activate(new_core, new_project, "Reliability")

## Initiate the dataflow
This indicates that the filtering and module / theme selections are complete

In [44]:
project_menu.initiate_dataflow(new_core, new_project)

2016-12-06 20:01:45,046 - INFO - aneris.control.data - New "hidden.dataflow_active" data stored with index JDJZGW
2016-12-06 20:01:45,046 - INFO - aneris.control.simulation - Datastate stored
2016-12-06 20:01:45,056 - INFO - aneris.control.simulation - Datastate with level "modules initial" stored


## Load test data

Prepare the test data for loading. The files required can be found in the test_data directory of the source code and should be copied to the directory that the notebook is running. When the python file is run a pickle file is generated containing a dictionary of inputs.

In [45]:
%run inputs_wp4.py
%run fixed_tidal_fixed_layout_reliability.py

generate test data: D:\Usuarios\108630\Programming\Python\notebooks\inputs_wp4.pkl
generate test data: D:\Usuarios\108630\Programming\Python\notebooks\fixed_tidal_fixed_layout_reliability.pkl


In [46]:
mooring_branch.read_test_data(new_core,
                              new_project,
                              "inputs_wp4.pkl")
mooring_branch.read_test_data(new_core,
                              new_project,
                              "fixed_tidal_fixed_layout_reliability.pkl")

2016-12-06 20:02:17,056 - INFO - aneris.control.data - New "farm.mean_wind_speed_100_year" data stored with index S7U9H6
2016-12-06 20:02:17,066 - INFO - aneris.control.data - New "device.system_profile" data stored with index 3DQ1A2
2016-12-06 20:02:17,066 - INFO - aneris.control.data - New "farm.max_water_level_50_year" data stored with index 73P7QZ
2016-12-06 20:02:17,066 - INFO - aneris.control.data - New "farm.mean_wind_direction_100_year" data stored with index KB16C6
2016-12-06 20:02:17,066 - INFO - aneris.control.data - New "device.umbilical_safety_factor" data stored with index 7RTQLL
2016-12-06 20:02:17,066 - INFO - aneris.control.data - New "device.turbine_diameter" data stored with index 5DVXWH
2016-12-06 20:02:17,066 - INFO - aneris.control.data - New "device.system_mass" data stored with index ABATK9
2016-12-06 20:02:17,076 - INFO - aneris.control.data - New "device.system_displaced_volume" data stored with index PL50DR
2016-12-06 20:02:17,076 - INFO - aneris.control.data

In [47]:
reliability_branch = pipe_tree.get_branch(new_core, new_project, "Reliability")
reliability_branch.read_test_data(new_core,
                                  new_project,
                                  "fixed_tidal_fixed_layout_reliability.pkl")

2016-12-06 20:02:20,217 - INFO - aneris.control.data - New "component.moorings_forerunner_CFR" data stored with index MKS1MR
2016-12-06 20:02:20,217 - INFO - aneris.control.data - New "component.moorings_swivel_NCFR" data stored with index H25O18
2016-12-06 20:02:20,226 - INFO - aneris.control.data - New "component.switchgear_NCFR" data stored with index CLF1AT
2016-12-06 20:02:20,226 - INFO - aneris.control.data - New "component.wet_mate_connectors_NCFR" data stored with index MFYM7X
2016-12-06 20:02:20,226 - INFO - aneris.control.data - New "project.lifetime" data stored with index 4YX4R1
2016-12-06 20:02:20,226 - INFO - aneris.control.data - New "farm.network_configuration" data stored with index 6PK7PE
2016-12-06 20:02:20,226 - INFO - aneris.control.data - New "component.foundations_pile_CFR" data stored with index FE5WSP
2016-12-06 20:02:20,226 - INFO - aneris.control.data - New "component.switchgear_CFR" data stored with index ST1WA6
2016-12-06 20:02:20,236 - INFO - aneris.contro

## Check if the module can be executed

In [48]:
can_execute = module_menu.is_executable(new_core, new_project, module_name)
display(can_execute)

True

In [49]:
input_status = mooring_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

In [50]:
input_status = reliability_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Execute the current module

The "current" module refers to the next module to be executed in the chain (pipeline) of modules. This command will only execute that module and another will be used for executing all of the modules at once.

Note, any data supplied by the module will be automatically copied into the active data state.

In [51]:
module_menu.execute_current(new_core, new_project)

2016-12-06 20:02:20,357 - INFO - aneris.control.simulation - Datastate with level "mooring and foundations start" stored
2016-12-06 20:02:22,617 - INFO - dtocean_moorings.main - ===================================================================
2016-12-06 20:02:22,617 - INFO - dtocean_moorings.main - Device ID: device001
2016-12-06 20:02:22,627 - INFO - dtocean_moorings.main - ===================================================================
2016-12-06 20:02:23,526 - WARNING - dtocean_moorings.core - WARNING: Reynolds number out of range for drag coefficient data
2016-12-06 20:02:23,526 - WARNING - dtocean_moorings.core - WARNING: Reynolds number out of range for drag coefficient data
2016-12-06 20:02:23,526 - WARNING - dtocean_moorings.core - WARNING: Reynolds number out of range for drag coefficient data
2016-12-06 20:02:23,526 - WARNING - dtocean_moorings.core - WARNING: Reynolds number out of range for drag coefficient data
2016-12-06 20:02:23,526 - WARNING - dtocean_moorings.co

## Examine the results

Currently, there is no robustness built into the core, so the assumption is that the module executed successfully. This will have to be improved towards deployment of the final software.

Let's check the number of devices and annual output of the farm, using just information in the data object.

In [52]:
output_status = mooring_branch.get_output_status(new_core, new_project)
message = html_dict(output_status)
HTML(message)

In [53]:
economics = mooring_branch.get_output_variable(new_core, new_project, "farm.moorings_foundations_economics_data")
economics.get_value(new_core, new_project)

,Key Identifier,Cost,Quantity,Year
0,id743,46595.722316,1.0,0.0
1,id472,1389.360000,1.0,0.0
2,id316,67.500000,1.0,0.0
3,id69,1015.154227,1.0,0.0
4,id316,67.500000,1.0,0.0
5,id422,2641.043412,1.0,0.0
6,id316,67.500000,1.0,0.0
7,id339,746.280000,1.0,0.0
8,id316,67.500000,1.0,0.0
9,id472,1389.360000,1.0,0.0


In [54]:
moorings = mooring_branch.get_output_variable(new_core, new_project, "farm.moorings_component_data")
moorings.get_value(new_core, new_project)

,Line Identifier,Marker
0,line000,0
1,line000,1
2,line000,2
3,line000,3
4,line000,4
5,line000,5
6,line000,6
7,line000,7
8,line001,8
9,line001,9


In [55]:
lines = mooring_branch.get_output_variable(new_core, new_project, "farm.line_data")
lines.get_value(new_core, new_project)

,Line Identifier,Type,Length,Dry Mass
0,line000,catenary,219.174306,1507.174176
1,line001,catenary,220.467777,1490.406841
2,line002,catenary,212.340890,1600.559112
3,line003,catenary,205.518546,1703.238915


In [56]:
umbilicals = mooring_branch.get_output_variable(new_core, new_project, "farm.umbilical_cable_data")
umbilicals.get_value(new_core, new_project)

,Key Identifier,Length,Dry Mass,Required Floatation,Floatation Length,Marker
0,id743,931.914446,10251.05891,151.02241,186.382889,0032


In [57]:
dimensions = mooring_branch.get_output_variable(new_core, new_project, "farm.moorings_dimensions")
dimensions.get_value(new_core, new_project)

,System Identifier,Footprint Area,Volume
0,subhub001,0.000000,0.000000e+00
1,device001,15707.963268,1.898742e+06
2,subhub002,0.000000,0.000000e+00
3,array,0.000000,0.000000e+00


In [58]:
network = mooring_branch.get_output_variable(new_core, new_project, "farm.moorings_foundations_network")
net_val = network.get_value(new_core, new_project)

In [59]:
import pprint
pprint.pprint(net_val)

{'nodes': {'array': {'Substation foundation': {'marker': [[41, 42]],
                                               'quantity': Counter({'id718': 1, 'grout': 1})}},
           'device001': {'Foundation': {'marker': [[33, 34],
                                                   [35, 36],
                                                   [37, 38],
                                                   [39, 40]],
                                        'quantity': Counter({'grout': 4, 'id516': 3, 'id528': 1})},
                         'Mooring system': {'marker': [[0,
                                                        1,
                                                        2,
                                                        3,
                                                        4,
                                                        5,
                                                        6,
                                                        7],
                 